In [1]:
%pwd

'd:\\SL\\End to End Projects\\New folder\\research'

In [2]:
import os

os.chdir("../")

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dirs: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
from mlproject.constants import *
from mlproject.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
    ):
         self.config = read_yaml(config_filepath)
         self.params = read_yaml(params_filepath)
         self.schema = read_yaml(schema_filepath)
         
         create_directories([self.config.artifacts_root])
    
    def data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dirs])
        
        data_ingestion_config = DataIngestionConfig(
            root_dirs = config.root_dirs,
            source_url = config.source_url,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        
        return data_ingestion_config

In [6]:
import os
import requests
import zipfile
from mlproject import logger
from mlproject.utils.common import get_size

In [7]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        """Download the file from source_url if it does not exist locally."""
        if not os.path.exists(self.config.local_data_file):
            print(self.config.local_data_file)
            print(self.config.source_url)
            response = requests.get(self.config.source_url, stream=True)
            if response.status_code == 200:
                with open(self.config.local_data_file, 'wb') as f:
                    f.write(response.content)
                print(f"{self.config.local_data_file} downloaded!")
            else:
                print(f"Failed to download file, status code: {response.status_code}")
        else:
            print(f"File already exists of size: {get_size(self.config.local_data_file)}")
    
    
    def extract_zip_file(self):
        """
        Extracts a ZIP file to the specified directory.

        Parameters:
        zip_file_path (str): The path to the ZIP file.
        extract_to_dir (str): The directory to extract the files to.

        Returns:
        None
        """
        zip_file_path = self.config.local_data_file
        extract_to_dir = self.config.unzip_dir
        # Check if the ZIP file exists
        print("zip file path:",zip_file_path)
        if not os.path.isfile(zip_file_path):
            raise FileNotFoundError(f"The file {zip_file_path} does not exist.")
        
        # Check if the extraction directory exists, if not, create it
        if not os.path.exists(extract_to_dir):
            os.makedirs(extract_to_dir)
        
        # Extract the ZIP file
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to_dir)

        print(f"Files extracted to {extract_to_dir}")

In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2024-07-27 22:42:12,552]: yaml file: config\config.yaml loaded successfully:
[2024-07-27 22:42:12,554]: yaml file: params.yaml loaded successfully:
[2024-07-27 22:42:12,555]: yaml file: schema.yaml loaded successfully:
Directory 'artifacts' created successfully or already exists.
[2024-07-27 22:42:12,556]: Created directory at: artifacts:
Directory 'artifacts/data_ingestion' created successfully or already exists.
[2024-07-27 22:42:12,556]: Created directory at: artifacts/data_ingestion:
artifacts/data_ingestion/monthly_milk_production.zip
https://github.com/sumairrasi/data/raw/main/monthly_milk_production.zip
artifacts/data_ingestion/monthly_milk_production.zip downloaded!
zip file path: artifacts/data_ingestion/monthly_milk_production.zip
Files extracted to artifacts/data_ingestion


In [ ]:
import os
import requests

local_data_file = 'artifacts/data_ingestion/monthly_milk_production.zip'
source_url = 'https://github.com/sumairrasi/data/raw/main/monthly_milk_production.zip'

def get_size(file_path: str) -> int:
    """Return the size of the file located at file_path."""
    return os.path.getsize(file_path)

def download_file():
    """Download the file from source_url if it does not exist locally."""
    if not os.path.exists(local_data_file):
        response = requests.get(source_url, stream=True)
        if response.status_code == 200:
            with open(local_data_file, 'wb') as f:
                f.write(response.content)
            print(f"{local_data_file} downloaded!")
        else:
            print(f"Failed to download file, status code: {response.status_code}")
    else:
        print(f"File already exists of size: {get_size(local_data_file)}")

download_file()
